# Watershed Delineation - Step-by-Step Example

This notebook demonstrates the functional approach to watershed delineation.
Each step can be run independently for easy debugging and verification.

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import networkx as nx

import graph_delineator.delineate as gd

In [2]:
# Define MERIT data directories
merit_dirs = {
    'catchment': Path('/nas/cee-ice/data/MERIT-BASINS/pfaf_level_02/'),
    'river': Path('/nas/cee-ice/data/MERIT-BASINS/pfaf_level_02/'),
    'flow_dir': Path('/nas/cee-ice/data/MERIT-Hydro/processed/dir/'),
    'accum': Path('/nas/cee-ice/data/MERIT-Hydro/processed/upg/'),
    'megabasins': Path('/nas/cee-ice/data/MERIT-Hydro/basins_level2/merit_hydro_vect_level2.shp')
}

# Load gauges
# gauges_csv = Path('/nas/cee-water/cjgleason/ted/graph_delineator/inputs/test.csv')
gauges_csv = Path('/nas/cee-water/cjgleason/ted/graph_delineator/inputs/test.csv')


In [3]:
results = gd.delineate_watershed(gauges_csv, merit_dirs)


GRAPH-BASED WATERSHED DELINEATION

Processing Basin 73 (contains 1 outlets)
Processing outlet USGS-01059000 (12 gauges)
  Inserted 12 gauges
  Completed 154 merges in 4 iterations


COMPLETE: Processed 1 watersheds


In [4]:
results

{'USGS-01059000': DelineationResult(graph=<networkx.classes.digraph.DiGraph object at 0x7e07b19e2780>, outlet_id='USGS-01059000')}

In [5]:
subbasins, gauges = results['USGS-01059000'].to_geodataframes()

In [9]:
subbasins.plot()

,id,geometry,area_km2,uparea_km2,node_type,is_gauge,nextdown
0,73000548,"POLYGON ((-71.13375 45.24371, -71.13375 45.243...",389.422565,389.422565,merged,False,73000916
1,73001371,"POLYGON ((-70.64708 44.28204, -70.64708 44.282...",434.733247,617.340209,merged,False,USGS-01059000
2,73001335,"POLYGON ((-71.23625 44.44958, -71.23626 44.449...",39.410111,3544.132847,split_downstream,False,73001334
3,73001334,"POLYGON ((-71.29624 44.40126, -71.29622 44.401...",35.505600,3579.638447,merged,False,73001333
4,73001333,"POLYGON ((-71.35292 44.37125, -71.35288 44.371...",71.336601,3650.975048,merged,False,73001332
5,73001332,"POLYGON ((-71.27179 44.3304, -71.27171 44.3304...",167.494293,3886.660205,merged,False,73001331
6,73001331,"POLYGON ((-71.05458 44.45375, -71.05458 44.453...",58.471382,3945.131587,merged,False,73001330
7,73001330,"POLYGON ((-70.98208 44.39049, -70.98209 44.390...",65.513601,4191.020442,merged,False,73001329
8,73001329,"POLYGON ((-70.93792 44.32708, -70.93788 44.327...",134.476045,4325.496488,merged,False,73001328
9,73001328,"POLYGON ((-70.81625 44.44792, -70.81621 44.447...",93.089593,4418.586080,merged,False,73001327
